In [286]:
import sys
import re

In [287]:
#Step 1. Get the list of words with i from a sample corpus

def WordsWithI():
    
    file = open('sample_corpus.txt', 'r', encoding='utf-8')
    #words = re.findall('[^\s]*i(?!werty)[^\s]*', file) #[^\s]*q(?!werty)[^\s]*
    #words = re.findall('ˆ|\s([a-zA-Zа-яА-ЯіїЇєЄґҐ]*i|I|і|ї[a-zа-яіїєґ]*)[.?,!"()]|\s|$', file) 
    words = re.findall(r'[^ ]*[і][^ ]+', file.read())
    
    f = open('words with i', 'w', encoding='utf-8')
    
    for word in words:
        f.write('%s\t\n'% (word))
    f.close()
    file.close()
    
WordsWithI()

#Step 2. Handwork - put the equivalents in Russian and rename it to 'project_lexics_2'

In [288]:
#Step 3. Make an aestetic work, using HW 2-4 (see 1-3 in goals). 

#Dictionaries Maker from hand-based analytics of the sample corpus.


def Dictionaries():
    
#Dictionary 1 with letters, morphemes, phonetic features to change:
    letters_phon_morph = open('project_letters.txt', 'r', encoding='utf-8')
    let_ph_mo = {}
    for line in letters_phon_morph:
        line = line.replace('\n', '')
        line = line.split('\t')
        let_ph_mo[line[0]] = line[1]
    
#Dictionary 2 with short lexems:    
    lexics = open('project_lexics.txt', 'r', encoding='utf-8')
    lexic = {}
    for line in lexics:
        line = line.replace('\n', '')
        line = line.split('\t')
        lexic[line[0]] = line[1]

#Dictionary 3 based on Step 1:  
    lexics_2 = open('project_lexics_2.txt', 'r', encoding='utf-8')
    lexic2 = {}
    for line in lexics_2:
        line = line.replace('\n', '')
        line = line.split('\t')
        lexic2[line[0]] = line[1]
        
Dictionaries()

In [290]:
#'Cosmetic' code: changes - phonetical, morhological and small lexical features

def Step3():
    
    file = open('sample_corpus.txt', 'r', encoding='utf-8') #actually we can use wiki.txt here
    f = open('cosmetic_wiki.txt', 'w', encoding='utf-8')

    Dictionaries()
    for word in file: #rename as a line
        if word.strip() == '': continue
        for key, value in lexic.items():
            #print(word, key, value)
            word = re.sub(' '+ key + ' ', ' ' + value + ' ', word) #я хочу, чтобы менялось только полностью слово
        #for key, value in lexic2.items(): 
            #project_lexics_2.txtword = re.sub(key, value, word)
        word = word.strip().split(' ') #rename tokens
        for w in word:
            if w.strip() == '': continue
            tr = w
            for key, value in let_ph_mo.items():
                tr = tr.replace(key, value)

            f.write((tr) + ' ')
            #print(tr)
        #print()
    file.close()
    f.close()
    
Step3()


In [291]:
#Step 4. Train part. HW 4

#Преобразовать слово внутри списка в список, запомнить букву в списке по индексу и на этом же месте найти соответсвующую русскую букву
#Ненужные буквы удалить, чтобы осталось только руссинские i и их русские эквиваленты
#На выходе получится файл

#mostly lucc4's code
def Preparation():
    
    lines = open('project_lexics_2.txt', 'r', encoding='utf-8')
    f = open('train.txt', 'w', encoding='utf-8')
    
    tempArr = []

    for line in lines:
        curline = re.sub('\n', '', line)
        words = curline.split('\t')
        lineArr = []
        for word in words:
            letters = []
            for letter in word:
                letters.append(letter)
            lineArr.append(letters);
        tempArr.append(lineArr)                      
    lines.close()
    
    
    for wordArr in tempArr:
        letterPos = -1;
        for x in range(0, len(wordArr[0])):
            if wordArr[0][x] == 'і':
                letterPos = x;
                break;
        if len(wordArr[1])-1 < letterPos:
            secondLetter= '!'
        else:
            secondLetter = wordArr[1][letterPos]
        f.write('і\t' + secondLetter + '\n')    
        
    f.close()

Preparation()

In [293]:
#adobted spectei's codes

def Train():
    
    # read all of the input lines in our trained meanings
    lines = open('train.txt', 'r', encoding='utf-8')
    f = open('project_model.tsv', 'w', encoding='utf-8')

    meaning_count = {}
    letters = {}
    letters_of_letter = {}
    total = 0

    for line in lines:
        if '\t' not in line:
            continue
        row = line.strip().split('\t')
        
        meaning = row[1]

        # if we haven't seen the meaning before then initialise the count to 0
        if meaning not in meaning_count:
            meaning_count[meaning] = 0
        meaning_count[meaning] = meaning_count[meaning] + 1

        letter = row[0]
        
        if letter not in letters_of_letter: 
            letters_of_letter[letter] = 0
        letters_of_letter[letter] = letters_of_letter[letter] + 1
        
        if letter not in letters:
            letters[letter] = {}

        if meaning not in letters[letter]:
            letters[letter][meaning] = 0
        letters[letter][meaning] = letters[letter][meaning] + 1
        total = total + 1
        #print(meaning_count[meaning], meaning, row)
    # {'ADJ': 2, 'NOUN': 5, 'NUM': 2, '_': 80, 'PROPN': 3}                                                       

    for meaning in meaning_count:
        freq = meaning_count[meaning]/total
        #print('%.2f\t%d\t%s\t%s' % (freq, meaning_count[meaning], meaning, '_'))

    # for each of the words in the matrix
    for letter in letters:
        # for each of the tag
        for meaning in letters[letter]:
            freq = letters[letter][meaning]/letters_of_letter[letter]
            f.write('%.2f\t%d\t%s\t%s\n' % (freq, letters[letter][meaning], meaning, letter))
            print('%.2f\t'% (freq) + '%d\t'% (letters[letter][meaning]) + '%s\t'% (meaning) + '%s'%(letter))
            #print('%.2f\t%d\t%s\t%s\n' % (freq, letters[letter][meaning], meaning, letter))


    f.close()        
    lines.close()
Train()

0.08	3	n	і
0.49	19	и	і
0.03	1	"	і
0.03	1	в	і
0.05	2	о	і
0.15	6	е	і
0.03	1	ь	і
0.05	2	х	і
0.03	1	d	і
0.03	1	м	і
0.03	1	»	і
0.03	1	я	і


In [294]:
#Step 5. Test. HW 5

#adopted spectei's code

def Test():

    fd = open('project_model.tsv', 'r', encoding='utf-8')


    frequent_meaning = 'X' # variable to store the most frequent tag we have seen
    last_max = 0 # variable to store the maximum frequency we have seen

    letter_meaning = {}  # word_tag['cat'] = 'NOUN'
    letter_prob = {} # word_prob['cat'] = 0.6

    for line in fd:
        line = line.strip() 
        line = line.split('\t') 
        # ['0.17', '3', 'PROPN', '_']
        if int(line[1]) > last_max: 
            last_max = int(line[1]) 
            frequent_meaning = line[2]

        # if we have never seen the surface form before, the most frequent tag is the one we just saw
        #print(line)
        surface_form = line[3] # this is the surface form
        prob = float(line[0]) # this is the probability
        meaning = line[2] # this is the tag
        if surface_form not in letter_meaning: 
            letter_meaning[surface_form] = meaning 
            letter_prob[surface_form] = prob

        # if we have seen the surface form before
        if surface_form in letter_meaning:
            # if the current probability is higher than the one we saw before
            if prob > letter_prob[surface_form]:
                letter_meaning[surface_form] = meaning
                letter_prob[surface_form] = prob

    # read the input and assign the most frequent tag to each token

    read = open('cosmetic_wiki.txt', 'r', encoding='utf-8')
    f = open('Russinized_corpus.txt', 'w', encoding='utf-8')

    #print(letter_meaning)
    for line in read:
        line = line.strip().split(' ')
        #print(line)
        for w in line:
            if w.strip() == '': continue
            #print(w)
            chars = [c for c in w]
            for j in range(0, len(chars)):
                if chars[j] == 'і':
                    chars[j] = frequent_meaning
                
            w = (''.join(chars))     
            print(w)
            f.write((w) + ' ')
                #letter['i'] = frequent_meaning
                
                #print(let)
                #print('\t'.join(letter))
                #f.write('\t'.join(line))
                

        print()

    read.close()
    f.close()
    fd.close()

Test()

Led
Zeppelin
(Лед
Зеппелин,
походить
от
ангхл.
"Lead
Zeppelin"
—
«оловяна
воздухолодь»)
—
британская
роковая
гхрупа
что
ся
тримле
за
едну
зо
закладателїв
гард-року.
Уж
30
роков
по
роспадї
с
причины
смерти
бубеника
Джона
Бонама
в
1980,
ся
Led
Zeppelin
дость
высоко
оцїнюють
за
умелескы
досягнутя,
комерчный
успих
и
шырокый
вплив.
ГХрупа
продала
понад
300
милионов
албумов
на
свитї,
в
том
числї
111.5
милионов
в
США,
вшыткы
їх
штудиёвы
албумы,
что
не
были
компилациями,
были
в
Top
10
журналу
Billboard
(США)
.
Led
Zeppelin
суть
першым
в
списку
100
найвызначнїшых
гард-роковых
капел
от
VH1.
Істория.
По
послїднїм
розпадї
гхрупы
«The
Yardbirds»
7
юла
1968
року
гхитариста
Джомми
Пейдж
мусив
зробити
нове
зложиня
гхрупы.
На
тот
час
ся
с
Пейджом
споёвала
репутация
жаданого
гхитаристу.
Вин
уж
грав
как
штудиовый
музикант
в
такых
гхрупах,
как
The
Who,
The
Kinks,
Donovan
и
много
иншых.
Другым
до
нового
зложиня
быв
закликаный
бас-гхитариста
Джон
Пол
Джонс
(Пейдж
ся
с
ним
познакомив
в
априлю
1968
року
в
час